# Tensorflow - TensorBoard

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

C:\Users\Doosik\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Doosik\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Doosik\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Doosik\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: P

### Reinstall protobuf with C++ implementation

To prevent error in protobuf while running tensorboard, protobuf of C++ implementation should be reinstalled.
See https://github.com/tensorflow/tensorboard/issues/1566 for more information.

```bash
set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=cpp
pip install --upgrade --force-reinstall protobuf==3.9.0
```

In [38]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [39]:
def weight_variable(shape):
    """Create a weight variable with appropriate initialization."""
    initial = tf.random.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    """Create a bias variable with appropriate initialization."""
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def variable_summaries(var):
    """Attach a lot of summaries to a Tensor (for TensorBoard visualization)."""
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(input_tensor=var)
        
        # Add summaries.
        tf.summary.scalar('mean', mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(input_tensor=tf.square(var - mean)))
        tf.summary.scalar('stddev', stddev)
        tf.summary.scalar('max', tf.reduce_max(input_tensor=var))
        tf.summary.scalar('min', tf.reduce_min(input_tensor=var))
        tf.summary.histogram('histogram', var)

In [40]:
def nn_layer(input_tensor, input_dim, output_dim, layer_name, act=tf.nn.relu):
    """Reusable code for making a simple neural net layer.

    It does a matrix multiply, bias add, and then uses ReLU to nonlinearize.
    It also sets up name scoping so that the resultant graph is easy to read,
    and adds a number of summary ops.
    """
    # Adding a name scope ensures logical grouping of the layers in the graph.
    with tf.name_scope(layer_name):
        # This Variable will hold the state of the weights for the layer
        with tf.name_scope('weights'):
            weights = weight_variable([input_dim, output_dim])
            variable_summaries(weights)  # Add weights summary.
        with tf.name_scope('biases'):
            biases = bias_variable([output_dim])
            variable_summaries(biases)  # Add biases summary.
        with tf.name_scope('Wx_plus_b'):
            preactivate = tf.matmul(input_tensor, weights) + biases
            tf.summary.histogram('pre_activations', preactivate)  # Add histogram summary.
        activations = act(preactivate, name='activation')
        tf.summary.histogram('activations', activations)  # Add histogram summary.
        return activations

In [41]:
# Build model.
def train(data_dir, fake_data, max_steps, learning_rate, dropout, log_dir):
    # Import data
    mnist = input_data.read_data_sets(data_dir, fake_data=fake_data)

    sess = tf.InteractiveSession()

    # Create a multilayer model.

    # Input placeholders
    with tf.name_scope('input'):
        x = tf.placeholder(tf.float32, [None, 784], name='x-input')
        y_ = tf.placeholder(tf.int64, [None], name='y-input')

    with tf.name_scope('input_reshape'):
        image_shaped_input = tf.reshape(x, [-1, 28, 28, 1])
        tf.summary.image('input', image_shaped_input, max_outputs=10)  # Add image to summary

    hidden1 = nn_layer(x, 784, 500, layer_name='layer1')

    with tf.name_scope('dropout'):
        keep_prob = tf.placeholder(tf.float32)
        tf.summary.scalar('dropout_keep_probability', keep_prob)
        dropped = tf.nn.dropout(hidden1, keep_prob=keep_prob)

    # Do not apply softmax activation yet, see below.
    y = nn_layer(dropped, 500, 10, layer_name='layer2', act=tf.identity)

    with tf.name_scope('cross_entropy'):
        # The raw formulation of cross-entropy,
        #
        # tf.reduce_mean(-tf.reduce_sum(y_ * tf.math.log(tf.softmax(y)), reduction_indices=[1]))
        #
        # can be numerically unstable.
        #
        # So here we use tf.losses.sparse_softmax_cross_entropy on the
        # raw logit outputs of the nn_layer above, and then average across
        # the batch.
        with tf.name_scope('total'):
            cross_entropy = tf.losses.sparse_softmax_cross_entropy(
                    labels=y_, logits=y)
    tf.summary.scalar('cross_entropy', cross_entropy)

    with tf.name_scope('train'):
        train_step = tf.train.AdamOptimizer(learning_rate).minimize(
            cross_entropy)

    with tf.name_scope('accuracy'):
        with tf.name_scope('correct_prediction'):
            correct_prediction = tf.equal(tf.argmax(input=y, axis=1), y_)
        with tf.name_scope('accuracy'):
            accuracy = tf.reduce_mean(input_tensor=tf.cast(correct_prediction, tf.float32))
    tf.summary.scalar('accuracy', accuracy)

    # Merge all the summaries and write them out to
    # /tmp/tensorflow/mnist/logs/mnist_with_summaries (by default)
    merged = tf.summary.merge_all()
    train_writer = tf.summary.FileWriter(log_dir + '/train', sess.graph)
    test_writer = tf.summary.FileWriter(log_dir + '/test')
    tf.global_variables_initializer().run()

    # Train the model, and also write summaries.
    # Every 10th step, measure test-set accuracy, and write test summaries
    # All other steps, run train_step on training data, & add training summaries

    def feed_dict(train):
        """Make a TensorFlow feed_dict: maps data onto Tensor placeholders."""
        if train or fake_data:
            xs, ys = mnist.train.next_batch(100, fake_data=fake_data)
            k = dropout
        else:
            xs, ys = mnist.test.images, mnist.test.labels
            k = 1.0
        return {x: xs, y_: ys, keep_prob: k}

    for i in range(max_steps):
        if i % 10 == 0:    # Record summaries and test-set accuracy
            summary, acc = sess.run([merged, accuracy], feed_dict=feed_dict(False))
            test_writer.add_summary(summary, i)
            print('Accuracy at step %s: %s' % (i, acc))
        else:    # Record train set summaries, and train
            if i % 100 == 99:    # Record execution stats
                run_options = tf.RunOptions(
                        trace_level=tf.RunOptions.FULL_TRACE)
                run_metadata = tf.RunMetadata()
                summary, _ = sess.run([merged, train_step],
                                      feed_dict=feed_dict(True),
                                      options=run_options,
                                      run_metadata=run_metadata)
                train_writer.add_run_metadata(run_metadata, 'step%03d' % i)
                train_writer.add_summary(summary, i)
                print('Adding run metadata for', i)
            else:    # Record a summary
                summary, _ = sess.run([merged, train_step], feed_dict=feed_dict(True))
                train_writer.add_summary(summary, i)
    train_writer.close()
    test_writer.close()

In [42]:
# Prepare MNIST.
image_files = tf.keras.utils.get_file(
    "train-images-idx3-ubyte.gz",
    "http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz")
label_files = tf.keras.utils.get_file(
    "train-labels-idx1-ubyte.gz",
    "http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz")

In [43]:
import os

data_dir = os.path.split(image_files)[0]
fake_data = False
max_steps = 1000
learning_rate = 0.001
dropout = 0.9
log_dir = '../temp/tensorboard'

# Start training.
with tf.Graph().as_default():
    train(data_dir, fake_data, max_steps, learning_rate, dropout, log_dir)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /home/doosik/.keras/datasets/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /home/doosik/.keras/datasets/train-labels-idx1-ubyte.gz
Extracting /home/doosik/.keras/datasets/t10k-images-idx3-ubyte.gz
Extracting /home/doosik/.keras/datasets/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Accuracy at step 0: 0.0786
Accuracy at step 10: 0.7074
Accuracy at step 20: 0.8325
Accuracy at step 30: 0.846
Accuracy at step 40: 0.8822
Accuracy at step 50: 0.8871
Accuracy at step 60: 0.9021
Accuracy at step 70: 0.9098
Accuracy at step 80: 0.9149
Accuracy at step 90: 0.915
Adding run me

### Run tensorboard
```bash
tensorboard --logdir=./temp/tensorboard

```
### Open web browser (with Chrome or FireFox)
Microsoft Internet Explorer and Microsoft Edge browser does not work correctly!
- http://localhost:6006/

## Example: Linear Regression Problem (revised for TensorBoard)

In [44]:
import numpy as np
import tensorflow as tf

In [74]:
# Genrating random linear data
# There will be 50 data points ranging from 0 to 50
x = np.linspace(0, 50, 50)
y = np.linspace(20, 50, 50)

# Adding noise to the random linear data
x += np.random.uniform(-4, 4, 50)
y += np.random.uniform(-4, 4, 50)
n = len(x) # Number of data points

In [92]:
tf.reset_default_graph()

learning_rate = 0.01
training_epochs = 500

In [93]:
# Start the Tensorflow Session.
sess = tf.Session()

with tf.name_scope('linear_regression'):
    with tf.name_scope('input'):
        X = tf.placeholder(tf.float32, name="X")
    with tf.name_scope('output'):
        Y = tf.placeholder(tf.float32, name="Y")
    with tf.name_scope('learning_parameters'):
        W = tf.Variable(np.random.randn(), name="W")
        tf.summary.scalar('W', W)
        b = tf.Variable(np.random.randn(), name="b")
        tf.summary.scalar('b', b)
    with tf.name_scope('linear_function'):
        y_pred = X * W + b
    with tf.name_scope('cost_function'):
        cost = tf.reduce_sum(tf.pow(y_pred - Y, 2)) / (2 * n)
        tf.summary.scalar('cost', cost)

optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
merged = tf.summary.merge_all()
init = tf.global_variables_initializer()
sess.run(init)

In [94]:
with tf.summary.FileWriter( '../temp/linear', sess.graph) as train_writer:
    # Iterate through all the epochs.
    for epoch in range(training_epochs):
        # Feed each data point into the optimizer using Feed Dictionary.
        for (_x, _y) in zip(x, y):
            sess.run(optimizer, feed_dict={X: _x, Y: _y})

        # Display the result after every 50 epochs.
        if (epoch + 1) % 50 == 0:
            # Calculate the cost.
            [summary, c] = sess.run([merged, cost], feed_dict={X: _x, Y: _y})
            print("Epoch", (epoch + 1), ": cost =", c, "W =", sess.run(W), "b =", sess.run(b))
            train_writer.add_summary(summary, epoch)  # Write summary.

print(sess.run(W), sess.run(b))
sess.close()

Epoch 50 : cost = 0.00014310163 W = 0.9094262 b = 4.9180245
Epoch 100 : cost = 0.0001637479 W = 0.8711955 b = 6.7411556
Epoch 150 : cost = 0.00018325623 W = 0.8371787 b = 8.363329
Epoch 200 : cost = 0.00020157015 W = 0.80691093 b = 9.806725
Epoch 250 : cost = 0.00021859632 W = 0.77998 b = 11.090996
Epoch 300 : cost = 0.00023431146 W = 0.7560177 b = 12.233698
Epoch 350 : cost = 0.00024876377 W = 0.7346968 b = 13.250439
Epoch 400 : cost = 0.00026197883 W = 0.7157259 b = 14.155114
Epoch 450 : cost = 0.0002740309 W = 0.69884634 b = 14.960059
Epoch 500 : cost = 0.00028497423 W = 0.68382716 b = 15.676287
0.68382716 15.676287


### Run tensorboard
```bash
tensorboard --logdir=./temp/linear

```
### Open web browser (with Chrome or FireFox)
Microsoft Internet Explorer and Microsoft Edge browser does not work correctly!

- http://localhost:6006/

## Reference
- https://www.tensorflow.org/guide/summaries_and_tensorboard
- https://jhui.github.io/2017/03/12/TensorBoard-visualize-your-learning/